# zib gereed maken voor mapping

* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [1]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2024)

In [2]:
base = 'zibs2024'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['Nl.zorg.Alert-v5.0(2024EN).xlsx',
 'Nl.zorg.HealthcareProvider-v3.6(2024EN).xlsx',
 'Nl.zorg.HealthProfessional-v4.0(2024EN).xlsx',
 'Nl.zorg.MedicalDevice-v5.0(2024EN).xlsx',
 'Nl.zorg.part.AddressInformation-v1.2(2024EN).xlsx',
 'Nl.zorg.part.AnatomicalLocation-v1.0.4(2024EN).xlsx',
 'Nl.zorg.part.ContactInformation-v1.3.1(2024EN).xlsx',
 'Nl.zorg.part.NameInformation-v1.2(2024EN).xlsx',
 'Nl.zorg.Patient-v4.3(2024EN).xlsx',
 'Nl.zorg.Procedure-v6.0(2024EN).xlsx']

## kies een zib

In [9]:
file = files[5]
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.part.AnatomicalLocation-v1.0.4(2024EN).xlsx


C:\Users\edelman_nict1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,AnatomicalLocation,NaN,NaN,NaN,NaN,NaN,NL: AnatomischeLocatie,NaN,NaN,rootconcept,NL-CM:20.7.1,Root concept of the AnatomicalLocation partial...,SNOMED CT: 123037004 Body structure,NaN,NaN
1,NaN,NaN,Location,NaN,NaN,NaN,NaN,NL: Locatie,CD,0..1,data,NL-CM:20.7.4,Location on/in the body.,SNOMED CT: 363698007 Finding site,LocationCodelist,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LocationICD-O-3Codelist,NaN
3,NaN,NaN,Laterality,NaN,NaN,NaN,NaN,NL: Lateraliteit,CD,0..1,data,NL-CM:20.7.3,Laterality adds information about the body sid...,SNOMED CT: 272741003 Laterality,LateralityCodelist,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* kolommen opschonen tot benodigde

Handmatig:
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)

In [10]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,AnatomicalLocation,AnatomicalLocation,NL: AnatomischeLocatie,,,rootconcept,NL-CM:20.7.1,Root concept of the AnatomicalLocation partial...,SNOMED CT: 123037004 Body structure
1,Location,AnatomicalLocation.Location,NL: Locatie,CD,0..1,data,NL-CM:20.7.4,Location on/in the body.,SNOMED CT: 363698007 Finding site
3,Laterality,AnatomicalLocation.Laterality,NL: Lateraliteit,CD,0..1,data,NL-CM:20.7.3,Laterality adds information about the body sid...,SNOMED CT: 272741003 Laterality


## Excel opslaan

In [11]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)